In [34]:
import re
import requests
import urllib
import pandas as pd
import difflib
from statistics import mean

filename = "scPDB_Proteins_BindingSites.txt"
doc = open(filename, "r")
Lines = doc.readlines()

In [50]:
amino_acids = ['VAL', 'ILE', 'LEU', 'GLU', 'GLN', 'ASP', 'ASN', 'HIS', 'TRP', 'PHE', 'TYR',  'ARG', 'LYS', 'SER', 'THR', 'MET', 'ALA', 'GLY', 'PRO', 'CYS']

## scPDB

In [51]:
scPDB = {}
for i in range(len(Lines)):
    words = Lines[i].split('||')
    # we store only the first 4 letters of the first word which is the PDB id 
    idPDB = words[0][:4]
    # for the rest of the words we store only the positions on the animoacids so we ignore the first 3 letters
    positions = []
    for j in range(1,len(words)-1):
#         positions.append(words[j][3:6])
        if words[j][:3] in amino_acids:
            res = ''.join(filter(lambda i: i.isdigit(), words[j]))
            positions.append(res)
    # we store the list with the positions in a dictionary, idPDB will be the key 
    scPDB[idPDB] = [ int(x) for x in positions ]

## Similarity predictions with scPDB

In [59]:
import os
# loading all prediction files
files_pred = os.listdir('test_output/predictions/')
total_files = len(files_pred)
counter = 1
total_max_sim = []

In [60]:
def calc_similarity(l1,l2):
    count_com = 0
    for item in l1:
        if item in l2:
            count_com +=1
    return count_com/len(l2)*100

In [61]:
with os.scandir('test_output/predictions/') as entries:
    for entry in entries:
        
        # just a counter to track the iteration we are and be less impatient
        print(str(counter)+"/"+str(total_files),end="\r")
        counter += 1
        
        # we read the all pockets as dataframe
        p = pd.read_csv('test_output/predictions/'+entry.name)
        
        # we will keep the maximum similarity, so we need a max value to store it 
        max_sim = 0
        
        # we retrieve the pocket that is stored in scPDB
        idPDB = entry.name[:4]
        pocket_scPDB = scPDB[idPDB]
        
        # we read all pockets one by one and check the similarity with the pocket stored in scPDB
        for index, row in p.iterrows() :
            pocket = row[" residue_ids"]
            pocket_list = pocket.split()
            pocket_list = [ ''.join(filter(lambda i: i.isdigit(), x)) for x in pocket_list ]
            pocket_list =  [ int(x) for x in pocket_list ]
            simil = calc_similarity(pocket_list,pocket_scPDB)
            if simil>max_sim:
                max_sim = simil
        total_max_sim.append(max_sim)

In [62]:
mean(total_max_sim)

71.54275842790325